# Batched ODBA

This script runs through a directory containing accelerometer files and extracts ODBA as a feature. 


In [2]:
#dependencies
import numpy
import pylab
import csv
import os

Tweak here to change directories etc

In [50]:
# where to find files
indir = "C:\\CloudProjects\\2019\\Research\\Projects\\OpenWild\\Deployments\\BadgerMay2019\\Tests\\output\\"
# where to put the processed data
outdir = "C:\\CloudProjects\\2020\\Research\\Projects\\OpenWild\\Tests\\BadgerODBA\\"
# what the inputfile looks like
inpattern = "ACC303_%d.csv"
# what the outputfile should be
outpattern = "ODBA_%d.csv"
# ODBA is typically computed over a 2 second window, but this is a fairly arbitrary decision
windowsize = 50 

You should not need to touch anything below here - just run the whole notebook to process a directory

In [51]:
# Scan through a directory and extract everything that matches filepattern
# store in the accfiles list
filelist = os.listdir(indir)
accfiles = []
for f in filelist:
    # here we use the underscore '_' to ensure the file prefixes are the same - modify as needed
    if f.split('_')[0] == filepattern.split('_')[0]:
        accfiles.append(f)

In [64]:
def read_acc_csv(filename,directory):
    """This function reads a filename in a particular directory and returns all the accelerometer data as an array"""
    f = open(os.path.join(directory,filename),'r')
    reader = csv.DictReader(f)
    acc_array = []
    for row in reader:
        acc_vec = [float(row['x']),float(row['y']),float(row['z'])]
        acc_array.append(acc_vec)
    f.close()
    return numpy.array(acc_array)

In [69]:
def write_acc_csv(filename,directory,odba):
    """This function dumps an array of ODBA values to an output file"""
    # we have to set newline = '' because windows is funky
    f = open(os.path.join(directory,filename),'w',newline='')
    writer = csv.DictWriter(f,['ODBA'])
    writer.writeheader()
    for row in odba:
        writer.writerow({'ODBA':row})
    f.close()

In [70]:
def vec_to_odba(window):
    """This function takes in a window of accelerometer values and returns ODBA over the window"""
    X = window[:,0]
    Y = window[:,1]
    Z = window[:,2]
    numRows = len(X)
    # Simple moving average 
    ave_x = numpy.mean(X)
    ave_y = numpy.mean(Y)
    ave_z = numpy.mean(Z)
    # subtract the average from all the points in the window
    detrend_x = X-ave_x
    detrend_y = Y-ave_y
    detrend_z = Z-ave_z
    # calculate the ODBA value (1-norm)
    odba_value = numpy.mean(abs(detrend_x)) + numpy.mean(abs(detrend_y)) + numpy.mean(abs(detrend_z))
    # store into the ODBA array        
    return odba_value

In [71]:
def compute_odba(inputdata,windowlen=25):
    odba_array = []
    for idx in range(1,len(inputdata)-windowlen,windowlen):
        window = inputdata[idx:idx+windowlen,:]
        odba_val = vec_to_odba(window)
        odba_array.append(odba_val)
    return numpy.array(odba_array)

In [75]:
for f in accfiles:
    timestamp = os.path.splitext(f)[0].split('_')[1]
    outfile = outpattern%int(timestamp)
    print(f,"->",outfile)
    acc_array = read_acc_csv(f,indir)
    odba = compute_odba(acc_array,windowlen=windowsize)
    write_acc_csv(outfile,outdir,odba)
    
    
    

ACC303_0.csv -> ODBA_0.csv
ACC303_100.csv -> ODBA_100.csv
ACC303_300.csv -> ODBA_300.csv
ACC303_400.csv -> ODBA_400.csv
ACC303_500.csv -> ODBA_500.csv
ACC303_600.csv -> ODBA_600.csv
ACC303_700.csv -> ODBA_700.csv
ACC303_800.csv -> ODBA_800.csv
